# Weather Agent Demo

This notebook demonstrates a simple agent that can use a `get_weather` tool.

In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI client with OpenRouter settings
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENROUTER_API_KEY"),
)

## 1. Define the Tool

In [ ]:
def get_weather(city: str):
    """Gets the weather for a given city."""
    if "tokyo" in city.lower():
        return json.dumps({"city": "Tokyo", "temperature": "15 C", "conditions": "Cloudy"})
    elif "san francisco" in city.lower():
        return json.dumps({"city": "San Francisco", "temperature": "12 C", "conditions": "Foggy"})
    elif "paris" in city.lower():
        return json.dumps({"city": "Paris", "temperature": "18 C", "conditions": "Sunny"})
    else:
        return json.dumps({"city": city, "temperature": "unknown", "conditions": "unknown"})

## 2. Define the List of Tools

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get the current weather in a given city",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The city, e.g., San Francisco",
                    },
                },
                "required": ["city"],
            },
        },
    }
]

## 3. Initialize Conversation History

We'll initialize the message history with the system prompt. This cell can be re-run to reset the conversation.

In [ ]:
messages = [{"role": "system", "content": "You are a helpful assistant. When you need to get the current weather, you must use the 'get_weather' tool. For other questions, answer directly."}]

## 4. Define the Agent Logic

This function will take a user query, append it to the history, call the model, handle any tool calls, and return the final response.

In [ ]:
def run_conversation_turn(user_query: str):
    """Runs a single turn of the conversation."""
    global messages
    
    messages.append({"role": "user", "content": user_query})
    
    response = client.chat.completions.create(
        model="google/gemini-flash-1.5",
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    if tool_calls:
        available_functions = {"get_weather": get_weather}
        messages.append(response_message)
        
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(city=function_args.get("city"))
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        
        second_response = client.chat.completions.create(
            model="google/gemini-flash-1.5",
            messages=messages,
        )
        final_response_message = second_response.choices[0].message
        messages.append(final_response_message)
        return final_response_message.content
    else:
        messages.append(response_message)
        return response_message.content

## 5. Run the Agent

Now you can ask the agent questions. Execute the cell below with your query.

In [ ]:
user_query = "What is the weather like in Tokyo?"
assistant_response = run_conversation_turn(user_query)
print(f"Assistant: {assistant_response}")

In [ ]:
user_query = "What about in Paris?"
assistant_response = run_conversation_turn(user_query)
print(f"Assistant: {assistant_response}")